In [29]:
#import packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [30]:
#import data
df_crime = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/MergedData_2017-2021.csv')
df_covid_ward = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                              , sheet_name = 'Total Cases by Ward', engine='openpyxl')
df_covid_death = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                               , sheet_name = 'Lives Lost by Ward', engine='openpyxl')
df_sick_police = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/daily_count_police_out_due_to_covid.csv')


In [31]:
#delete columns not needed from crime data
del df_crime['REPORT_DAT']
del df_crime['BLOCK']
del df_crime['YBLOCK']
del df_crime['XBLOCK']
del df_crime['LATITUDE']
del df_crime['LONGITUDE']
del df_crime['END_DATE']

df_crime = df_crime.rename(columns = {'START_DATE':'date'})

df_crime['date'] = pd.to_datetime(df_crime['date'], utc=True).dt.date
df_crime['date'] = pd.to_datetime(df_crime['date'], utc=True)
df_crime.dtypes

SHIFT                    object
METHOD                   object
OFFENSE                  object
WARD                    float64
DISTRICT                float64
date        datetime64[ns, UTC]
dtype: object

In [32]:
df_crime.dtypes

SHIFT                    object
METHOD                   object
OFFENSE                  object
WARD                    float64
DISTRICT                float64
date        datetime64[ns, UTC]
dtype: object

In [33]:
#covid case data
df_covid_ward = df_covid_ward.transpose()
df_covid_ward.reset_index(drop = True, inplace= True)
df_covid_ward.columns=('date', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'unknown')
df_covid_ward = df_covid_ward.iloc[1: , :]
df_covid_ward['date'] = pd.to_datetime(df_covid_ward['date'], utc=True)
df_covid_ward['total_cases'] = (df_covid_ward['ward_1'] + df_covid_ward['ward_2'] + df_covid_ward['ward_3'] + df_covid_ward['ward_4'] + df_covid_ward['ward_5'] + df_covid_ward['ward_6'] +df_covid_ward['ward_7'] + df_covid_ward['ward_8'] +df_covid_ward['unknown'])
df_covid_ward['daily_cases'] = df_covid_ward['total_cases'].diff(1)

del df_covid_ward['ward_1']
del df_covid_ward['ward_2']
del df_covid_ward['ward_3']
del df_covid_ward['ward_4']
del df_covid_ward['ward_5']
del df_covid_ward['ward_6']
del df_covid_ward['ward_7']
del df_covid_ward['ward_8']
del df_covid_ward['unknown']


df_covid_ward.dtypes

date           datetime64[ns, UTC]
total_cases                 object
daily_cases                 object
dtype: object

In [34]:
#death data
df_covid_death = df_covid_death.transpose()
df_covid_death.reset_index(drop = True, inplace= True)
df_covid_death.columns=('date', 'all', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'Experienced Homelessness', 'unknown', 'na')
df_covid_death = df_covid_death.iloc[1: , :]
df_covid_death['date'] = pd.to_datetime(df_covid_death['date'], utc=True)
df_covid_death['daily_death'] = df_covid_death['all'].diff(1)

del df_covid_death['ward_1']
del df_covid_death['ward_2']
del df_covid_death['ward_3']
del df_covid_death['ward_4']
del df_covid_death['ward_5']
del df_covid_death['ward_6']
del df_covid_death['ward_7']
del df_covid_death['ward_8']
del df_covid_death['unknown']
del df_covid_death['na']
del df_covid_death['Experienced Homelessness']
del df_covid_death['all']

df_covid_death.dtypes

date           datetime64[ns, UTC]
daily_death                 object
dtype: object

In [35]:
#sick police
df_sick_police = df_sick_police.rename(columns = {'DATE_REPORTED':'date'})
df_sick_police['date'] = pd.to_datetime(df_sick_police['date'], utc=True)

df_sick_police.dtypes

date                             datetime64[ns, UTC]
OUT_POSITIVE_QUARANTINE_PSMPD                float64
dtype: object

In [36]:
merged = df_crime.merge(df_covid_ward, how='left', on = 'date').merge(df_covid_death, how='left', on = 'date').merge(df_sick_police, how='left', on = 'date')
merged.loc[merged['total_cases'] >1]

,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD
100787,EVENING,OTHERS,THEFT/OTHER,6.0,5.0,2020-09-01 00:00:00+00:00,14077,28,1,80.0
100788,EVENING,GUN,ASSAULT W/DANGEROUS WEAPON,8.0,7.0,2020-09-01 00:00:00+00:00,14077,28,1,80.0
100789,EVENING,OTHERS,THEFT/OTHER,4.0,4.0,2020-09-01 00:00:00+00:00,14077,28,1,80.0
100790,EVENING,OTHERS,THEFT/OTHER,6.0,1.0,2020-08-31 00:00:00+00:00,14049,57,0,87.0
100791,EVENING,OTHERS,THEFT F/AUTO,2.0,2.0,2020-09-01 00:00:00+00:00,14077,28,1,80.0
...,...,...,...,...,...,...,...,...,...,...
151064,DAY,OTHERS,BURGLARY,5.0,5.0,2021-02-04 00:00:00+00:00,37634,269,8,104.0
151065,DAY,OTHERS,THEFT F/AUTO,7.0,6.0,2021-02-04 00:00:00+00:00,37634,269,8,104.0
151066,DAY,OTHERS,THEFT F/AUTO,5.0,4.0,2021-02-03 00:00:00+00:00,37365,166,6,106.0
151067,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,8.0,7.0,2021-02-04 00:00:00+00:00,37634,269,8,104.0


In [37]:
merged3.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')

NameError: name 'merged3' is not defined